In [1]:
import pandas as pd


In [ ]:
df1=pd.read_csv('nptel_final.csv')

IOError: ignored

In [ ]:
from nltk.corpus import stopwords
#converting stop words to ascii to prevent unicode error and adding some new words that could intuitively be stop words here
vtt_content=df1.vtt_Content.tolist()
stop_words_ascii=[]
for each in stopwords.words('english'):
    stop_words_ascii.append(each.encode('ascii','ignore'))

stop_words_ascii.extend(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
stop_words_ascii.extend(['ah','uh','the','and','so','is','ok','um','ok','ha','language:','en'])

#print(stop_words_ascii)

#STEMMING

In [ ]:
import nltk
import string
import re

porter_stemmer = nltk.stem.porter.PorterStemmer()

def porter_tokenizer(text, stemmer=porter_stemmer):
    tokens = nltk.wordpunct_tokenize(text)
    stems = [porter_stemmer.stem(t) for t in tokens]
    no_punct = [s for s in stems if re.match('^[a-zA-Z]+$', s) is not None]
    return no_punct

In [ ]:
stemmed_vtt_content=[] 
#stemming each word for each row of vtt_description of videos
for eachcontent in vtt_content:
    eachcontent=eachcontent.lower().decode('utf-8')
    #print(type(eachcontent))
    tokenized=porter_tokenizer(eachcontent)
    preprocessed=' '.join(tokenized)
    preprocessed=preprocessed.encode('ascii','ignore')
    #print(preprocessed)
    stemmed_vtt_content.append(preprocessed)
  

In [ ]:
#removing stop words from stemmed vtt data
filtered_words_list=[]
i=0
for i in range(len(stemmed_vtt_content)):
    filtered_words=[]
    for item in stemmed_vtt_content[i].split(" "):
        if item not in stop_words_ascii:
            filtered_words.append(item)         
    #print(filtered_words)
    str1=" ".join(filtered_words)
    #print str1
    #print("\n NEXT DOCUMENT______________________________________________________________________",i)
    filtered_words_list.append(str1)
#print("FINAL_____________________________________________________________________",filtered_words_list)

In [ ]:
#checking the most often occuring words,utility function only for analysis 
import collections
for i in range(len(filtered_words_list)):
    #for list_1 in filtered_words_list[i].split(" "):
    counter = collections.Counter(filtered_words_list[i].split(" "))
    #print(counter.most_common())
    #print("______________________________________________________________________________________________")

#Applying TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = stop_words_ascii)

In [ ]:
tfidf_matrix =  tf.fit_transform(filtered_words_list)
feature_names = tf.get_feature_names() 

In [ ]:
len(feature_names)

14682

In [ ]:
tfidf_matrix

<902x14682 sparse matrix of type '<type 'numpy.float64'>'
	with 392333 stored elements in Compressed Sparse Row format>

In [ ]:
from pandas import DataFrame
doc_id=0
#showing 50 most important occuring words,as processed by tfidf
tfidf_words_list=[]
dense=tfidf_matrix.todense()
for i in range(len(filtered_words_list)):
    top_n=[]
    perVid_Transcript=dense[i].tolist()[0]
    phrase_scores= [pair for pair in zip(range(0, len(perVid_Transcript)), perVid_Transcript) if pair[1] > 0]
    sorted_phrase_scores=sorted(phrase_scores, key=lambda t: t[1] * -1)
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:50]:
        top_n.append(phrase.encode('ascii','ignore'))
        #print('{0} {1: <50} {2}'.format(i,phrase, score))
    tfidf_words_list.append(top_n)
    #print('\n Document {0} :{1}'.format(i,top_n))
#print('FINAL list of 50 words of all docs in one list',tfidf_words_list)

In [ ]:
#adding a new row in our dataframe of top tags with each vtt
df1['TFIDF_top_words']=tfidf_words_list
df1
df1.to_csv('nptel_tfidf_labelled.csv')

#print df1['TFIDF_top_words']